# Installation

In [2]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia -y

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12 jaxlib -f https://storage.googleapis.com/jax-releases/jax_releases.html
!pip install git+https://github.com/finetuneanon/transformers@gpt-j

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::pcre==8.45=h295c915_0
  - defaults/noarch::python-dateutil==2.8.2=pyhd3eb1b0_0
  - defaults/noarch::bleach==3.3.1=pyhd3eb1b0_0
  - defaults/linux-64::olefile==0.46=py37_0
  - defaults/noarch::jupyterlab_widgets==1.0.0=pyhd3eb1b0_1
  - defaults/linux-64::nbconvert==6.1.0=py37h06a4308_0
  - defaults/noarch::ipywidgets==7.6.3=pyhd3eb1b0_1
  - defaults/linux-64::importlib-metadata==3.10.0=py37h06a4308_0
  - defaults/noarch::testpath==0.5.0=pyhd3eb1b0_0
  - defaults/linux-64::lame==3.100=h7b6447c_0
  - defaults/linux-64::python==3.7.10=h12debd9_4
  - defaults/linux-64::gnutls==3.6.15=he1e5248_0
  - defaults/linux-64::ninja==1.10.2=hff7bd54_1
  - defaults/linux-64::bzip2==1.0.8=h7b6447c_0
  - defaults/linux-64::libxml2==2.9.12=h03d6c58_0
  - defaults/linux-64::pyqt==5.9.2=py37h05f1152_2
  - defaults/linux-64::gstreamer=

  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface-hub-0.0.8:
      Successfully uninstalled huggingface-hub-0.0.8
  Attempting uninstall: transformers
    Found existing installation: transformers 4.7.0.dev0
    Uninstalling transformers-4.7.0.dev0:
      Successfully uninstalled transformers-4.7.0.dev0
Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
Processing ./mesh-transformer-jax
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=24206 sha256=3d735a

## Get the actual GPT-J model [Source](https://github.com/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb)

In [3]:

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
#if you have issues install zstd: sudo apt install zstd 
!time tar -I zstd -xf step_383500_slim.tar.zstd

--2021-07-26 01:16:32--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9414712325 (8.8G) [application/octet-stream]
Saving to: ‘step_383500_slim.tar.zstd’

step_383500_slim.ta 100%[===================>]   8.77G  11.2MB/s    in 13m 33s 

2021-07-26 01:30:05 (11.0 MB/s) - ‘step_383500_slim.tar.zstd’ saved [9414712325/9414712325]


real	13m33.283s
user	0m12.213s
sys	0m38.907s

real	0m17.185s
user	0m11.773s
sys	0m11.127s


## Convert to torch [Source](https://gist.github.com/finetuneanon/ee196c6cd16af1de4ca444862414683a) Credit:[finetuneanon](https://github.com/finetuneanon)

In [4]:
!mkdir gpt-j-6B
!curl https://gist.githubusercontent.com/finetuneanon/a55bdb3f5881e361faef0e96e1d41f09/raw/e5a38dad34ff42bbad188afd5e4fdb2ab2eacb6d/gpt-j-6b.json > gpt-j-6B/config.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1334  100  1334    0     0   6771      0 --:--:-- --:--:-- --:--:--  6771


In [6]:
import torch
import numpy as np
import jax.numpy as jnp
import io
import os

torch.set_printoptions(linewidth=130, sci_mode=False)
np.set_printoptions(linewidth=130, suppress=True)

layers = 28
total_shards = 8
ckpt_dir = "step_383500/"
output_dir = "j6b_ckpt"

def reshard(x, old_shape):
    if len(x.shape) == 1:
        # print("epoch")
        # print(x)
        out = x[0:1]

    elif len(x.shape) == 2:
        #print(f"LN/bias {x.shape}")
        #print(x[:, :16])

        if (x[1:] == x[-1]).all():
            #print("LN")
            if (x[1:] == 0).all() or (x[1:] == 1).all():
                out = x[0:1]
            else:
                #print("shard bias")
                out = x[0:1] * 8#* x.shape[0] / old_shape[0]
        else:
            #print("bias")
            out = x.reshape(old_shape)

        #print(out[:, :16])

    elif len(x.shape) == 3:
        #print(f"weight {x.shape}")
        if x.shape[0] * x.shape[2] == old_shape[2]:
            #print("case 1")
            out = jnp.transpose(x, (1, 0, 2)).reshape(old_shape)
        elif x.shape[0] * x.shape[1] == old_shape[1]:
            #print("case 2")
            out = x.reshape(old_shape)
        else:
            raise Exception(f"unimplemented, {x.shape}, {old_shape}")
    else:
        raise Exception(f"unimplemented, {x}")
    #flattened, structure = jax.tree_flatten(out)
    #return flattened
    return out

def get_old_shape(t, dim=2):
    if len(t.shape) == 3:
        shard_shape = t.shape
        if dim == 1:
            return (shard_shape[0] * shard_shape[1], shard_shape[2])
        elif dim == 2:
            return (shard_shape[1], shard_shape[0] * shard_shape[2])
        else:
            raise ValueError(f"unsupported dim {dim}")
    if len(t.shape) == 2:
        return (t.shape[1] * t.shape[0],)
    else:
        raise ValueError(f"unsupported shape {t.shape}")

def read_shard(ckpt_dir):
    global part
    out = []
    idx = part
    file_path = ckpt_dir + f"{idx}.npz"
    #print(f"-- {file_path}")
    with open(file_path, "rb") as f:
        buf = f.read()
        f_io = io.BytesIO(buf)
        deserialized = np.load(f_io)
        for i in deserialized:
            out.append(deserialized[i])
            #print(deserialized[i].shape)
    return out

def save(ckpt):
    try: os.mkdir(output_dir)
    except: pass
    checkpoint = {}
    for i, x in enumerate(ckpt.items()):
        checkpoint[x[0]] = f"{output_dir}/b{i}.pt"
        torch.save(x[1], f"{output_dir}/b{i}.pt")
    torch.save(checkpoint, f"{output_dir}/m.pt")

unshard = None
transforms = [("transformer.wte.bias", None, None), ("transformer.wte.weight", unshard, 1)]

checkpoint = {}

layer_names = sorted(map(str, range(layers)))
for layer in layer_names:
    checkpoint[f"transformer.h.{layer}.attn.attention.bias"] = torch.tril(torch.ones(1, 1, 2048, 2048))
    checkpoint[f"transformer.h.{layer}.attn.attention.masked_bias"] = torch.tensor(-1e9)
    transforms.extend([
        (f"transformer.h.{layer}.attn.attention.q_proj.weight", unshard, 2),
        (f"transformer.h.{layer}.attn.attention.v_proj.weight", unshard, 2),
        (f"transformer.h.{layer}.attn.attention.k_proj.weight", unshard, 2),
        (f"transformer.h.{layer}.attn.attention.out_proj.weight", unshard, 1),
        (f"transformer.h.{layer}.mlp.c_fc.bias", unshard, 1),
        (f"transformer.h.{layer}.mlp.c_fc.weight", unshard, 2),
        (f"transformer.h.{layer}.mlp.c_proj.bias", None, None),
        (f"transformer.h.{layer}.mlp.c_proj.weight", unshard, 1),
        (f"transformer.h.{layer}.ln_1.bias", None, None),
        (f"transformer.h.{layer}.ln_1.weight", None, None),
    ])
transforms.extend([
    ("lm_head.bias", unshard, 1),
    ("lm_head.weight", unshard, 2),
    ("transformer.ln_f.bias", None, None),
    ("transformer.ln_f.weight", None, None),
])

part = 0
element = 0
while len(transforms) > 0:
    print(f"loading shards for part {part}")
    shards = list(map(read_shard, [f"{ckpt_dir}shard_{i}/" for i in range(total_shards)]))
    print(f"read from checkpoint")

    unsharded = []

    for all_shards in zip(*shards):
        x = np.stack(all_shards)
        # No idea why this is V2...?
        if x.dtype == np.dtype('V2'):
            x.dtype = jnp.bfloat16
        x = x.astype(np.float32)
        unsharded.append(x)
        #print(f"unsharded: {x.shape}")

    while len(transforms) > 0 and len(unsharded) > 0:
        transform = transforms.pop(0)
        params = unsharded.pop(0)
        if transform[2] is not None:
            old_shape = (1,) + get_old_shape(params, transform[2])
        else:
            old_shape = (params.shape[1],)
        print(f"< {params.shape} to {old_shape}")
        params = reshard(params, old_shape).squeeze(0).T
        params = torch.tensor(params.copy()).half()
        if params.isnan().any() or params.isinf().any():
            raise ValueError(f"fp16 over/underflow at {part} {element}")
        checkpoint[transform[0]] = params
        print(f"> {transform[0]} {params.shape}")
        element += 1
    part += 1

checkpoint['transformer.wte.weight'] = (checkpoint['transformer.wte.weight'].T + checkpoint['transformer.wte.bias'])
del checkpoint['transformer.wte.bias']

print(f"left over: {unsharded}")
print("saving")
torch.save(checkpoint, "./gpt-j-6B/pytorch_model.bin") # load as in: https://github.com/finetuneanon/misc/blob/main/SizeTest.ipynb
print("done")

loading shards for part 0
read from checkpoint


KeyboardInterrupt: 

In [8]:
import time
import torch
from transformers import GPTNeoForCausalLM, AutoConfig, GPT2Tokenizer
import torch
import transformers

In [9]:
#Will need at least 13-14GB of Vram
if torch.cuda.is_available():
    model = GPTNeoForCausalLM.from_pretrained("./gpt-j-6B").half().cuda()
else:
    model = GPTNeoForCausalLM.from_pretrained("./gpt-j-6B")

# Inference

In [11]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')

In [12]:
model.eval()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
        )
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=4096, out_features=16384, bias=True)
          (c_proj): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )


In [15]:
input_text = "Hello my name is Blake and"
input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

output = model.generate(
    input_ids,
    do_sample=True,
    max_length=20,
    top_p=0.7,
    top_k=0,
    temperature=1.0,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [16]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Hello my name is Blake and I am a 28 year old woman from Canada. I've been writing
